In [69]:
#import all packages
#-----------------------------------------------------
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import datetime
import plotly.express as px
import plotly.subplots
import plotly.graph_objects as go
from datetime import timedelta

#rename csv files
#-----------------------------------------------------
csv_ChargeRecords = "test.csv"

In [93]:
#read csv 
#----------------------------------------------------
df_ChargeRecords = pd.read_csv(csv_ChargeRecords, 
        header         = [0], 
        sep            = ',',
        parse_dates    = ['endtime', 'starttime']).rename(columns={
    'Hashed contract id':'UserID'
})

#change column type duration to timedelta
df_ChargeRecords["duration"] = pd.to_timedelta(df_ChargeRecords["duration"])

#move starttime to third column with pop and insert
start_time = df_ChargeRecords.pop("starttime")
df_ChargeRecords.insert(2, "starttime", start_time)

#change column type UserID to str
df_ChargeRecords["UserID"] = df_ChargeRecords["UserID"].astype(str)

#show data
df_ChargeRecords

,UserID,duration,starttime,endtime,connectorstandard,maxsocketpower,quantityinwh
0,877,03:14:31,2021-12-08 19:45:00,2021-12-08 23:00:00,IEC_62196_T2,22.0,25810
1,241,05:00:10,2021-12-08 18:53:00,2021-12-08 23:53:00,IEC_62196_T2,22.0,18320
2,544,00:01:04,2021-12-08 14:33:00,2021-12-08 14:34:00,IEC_62196_T2,22.0,0
3,241,02:03:36,2021-08-08 15:26:00,2021-08-08 17:30:00,IEC_62196_T2,22.0,7720
4,574,00:01:09,2021-06-08 09:31:00,2021-06-08 09:32:00,IEC_62196_T2,11.0,0
...,...,...,...,...,...,...,...
1994,988,01:50:47,2021-07-27 22:00:00,2021-07-27 23:51:00,IEC_62196_T2,22.0,6800
1995,988,01:42:09,2021-07-29 22:51:00,2021-07-30 00:33:00,IEC_62196_T2,22.0,5200
1996,988,01:46:00,2021-07-31 22:58:00,2021-01-08 00:44:00,IEC_62196_T2,22.0,6500
1997,988,01:45:06,2021-02-08 22:16:00,2021-03-08 00:01:00,IEC_62196_T2,22.0,6500


In [94]:
#check data type 
#----------------------------------------------------
df_ChargeRecords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   UserID             1999 non-null   object         
 1   duration           1999 non-null   timedelta64[ns]
 2   starttime          1999 non-null   datetime64[ns] 
 3   endtime            1999 non-null   datetime64[ns] 
 4   connectorstandard  1999 non-null   object         
 5   maxsocketpower     1999 non-null   float64        
 6   quantityinwh       1999 non-null   int64          
dtypes: datetime64[ns](2), float64(1), int64(1), object(2), timedelta64[ns](1)
memory usage: 109.4+ KB


In [95]:
#User specific details count
#-----------------------------------------------------
df_ChargeRecords_UserSpecCount = df_ChargeRecords.groupby(['UserID']).count().reset_index().drop([
    'duration','connectorstandard','maxsocketpower','quantityinwh'], axis=1).rename(columns={
    'endtime': 'NumberOfEndTime',
    'starttime': 'NumberOfStartTime'
})

#show data
df_ChargeRecords_UserSpecCount

,UserID,NumberOfStartTime,NumberOfEndTime
0,169,1,1
1,206,2,2
2,210,4,4
3,224,14,14
4,231,16,16
5,241,67,67
6,252,3,3
7,303,6,6
8,354,8,8
9,397,6,6


In [96]:
#check data type 
#----------------------------------------------------
df_ChargeRecords_UserSpecCount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   UserID             37 non-null     object
 1   NumberOfStartTime  37 non-null     int64 
 2   NumberOfEndTime    37 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1016.0+ bytes


In [97]:
#User specific details sum
#-----------------------------------------------------
df_ChargeRecords_UserSpecSum = df_ChargeRecords.groupby(['UserID']).agg({
    'duration': np.sum, 'quantityinwh': np.sum}).reset_index().rename(columns={
    'duration':'Duration',
    'quantityinwh':'ChargingEnergyinWh',
})
#cannot use sum() for timedelta dtype --> forcing sum with numpy

#show data
df_ChargeRecords_UserSpecSum

,UserID,Duration,ChargingEnergyinWh
0,169,0 days 00:20:52,23016
1,206,0 days 07:44:04,55135
2,210,0 days 01:40:45,5472
3,224,1 days 07:13:43,193848
4,231,0 days 07:26:54,463972
5,241,7 days 10:20:21,686010
6,252,0 days 09:14:11,27774
7,303,0 days 14:43:17,155173
8,354,0 days 02:19:08,32892
9,397,0 days 04:27:23,88860


In [98]:
#Common data --> merge all 
#------------------------------------------------------------
df_UserSpec = pd.merge(df_ChargeRecords_UserSpecCount,
                                  df_ChargeRecords_UserSpecSum,
                                  how='left',
                                  on=['UserID']
                                 )
#User ID as an index
#df_UserSpec = df_UserSpec.set_index('UserID')

#Proximation of charging behaviour --> Charging Energy / Number of Start
df_UserSpec['EnergyperCycle'] = df_UserSpec['ChargingEnergyinWh']/ df_UserSpec['NumberOfStartTime']

df_UserSpec

,UserID,NumberOfStartTime,NumberOfEndTime,Duration,ChargingEnergyinWh,EnergyperCycle
0,169,1,1,0 days 00:20:52,23016,23016.000000
1,206,2,2,0 days 07:44:04,55135,27567.500000
2,210,4,4,0 days 01:40:45,5472,1368.000000
3,224,14,14,1 days 07:13:43,193848,13846.285714
4,231,16,16,0 days 07:26:54,463972,28998.250000
5,241,67,67,7 days 10:20:21,686010,10238.955224
6,252,3,3,0 days 09:14:11,27774,9258.000000
7,303,6,6,0 days 14:43:17,155173,25862.166667
8,354,8,8,0 days 02:19:08,32892,4111.500000
9,397,6,6,0 days 04:27:23,88860,14810.000000


In [109]:
#visualization 
#-----------------------------------------------------
fig_UserSpec_perCycle = px.histogram(data_frame=df_UserSpec,
                                     x='EnergyperCycle',
                                     y='UserID',
                                     color='UserID',
                                     template="simple_white"
                                    )

fig_UserSpec_perCycle.show()